In [1]:
import requests
from urllib.parse import urlencode, quote

import pandas as pd
import api_keys

import polyline

In [2]:
base_url = 'https://graphhopper.com/api/1/route'

requests.get(base_url).json()

{'message': "No parameters specified, at least 'key' is necessary. Please register and see documentation: https://www.graphhopper.com/developers/"}

In [3]:
# curl "https://graphhopper.com/api/1/route?point=51.131,12.414&point=48.224,3.867&profile=car&locale=de&calc_points=false&key=api_key"

locations_df = pd.read_csv('datasets/station_information.csv', usecols=['station_id', 'lat', 'lon'])
locations_df

locations_df.head()

,station_id,lat,lon
0,7000,43.639832,-79.395954
1,7001,43.664964,-79.383550
2,7002,43.667131,-79.399555
3,7003,43.667018,-79.402796
4,7005,43.648001,-79.383177


In [62]:
def get_route_json(lat1, lon1, lat2, lon2):
    base_url = 'https://graphhopper.com/api/1/route'
    url = f'{base_url}?point={lat1},{lon1}&point={lat2},{lon2}&profile=bike&locale=en&calc_points=true&points_encoded=False&key={api_keys.api_key}'
    r = requests.get(url).json()
    print(r)
    try:
        pass
    except KeyError:
        pass
#return polyline.decode(r.get('paths')[0].get('points'))
    

In [55]:
import folium

lat_lon_df = pd.read_csv("datasets/lon-lat.csv", usecols=['lat_start', 'lon_start', 'lat_end', 'lon_end'])
lat_lon_df

,lat_start,lon_start,lat_end,lon_end
0,43.639444,-79.423611,43.639444,-79.423611
1,43.631400,-79.430400,43.639278,-79.411574
2,43.658938,-79.383518,43.656026,-79.385327
3,43.635492,-79.398253,43.675791,-79.406980
4,43.649172,-79.444509,43.639700,-79.448650
...,...,...,...,...
7486,43.640978,-79.376785,43.632244,-79.410810
7487,43.645469,-79.411084,43.646218,-79.385505
7488,43.671389,-79.382919,43.664964,-79.383550
7489,43.655431,-79.380653,43.645572,-79.393990


In [56]:
# Create a base map (centered at the first coordinate)

m = folium.Map(location=[lat_lon_df["lat_start"].mean(), lat_lon_df["lon_start"].mean()], zoom_start=5)


In [63]:
def add_marker(lat1, lon1, lat2, lon2):
    folium.Marker(
        location=[lat1, lon1],
        popup="Start Location",
        tooltip="Start",
        icon=folium.Icon(color="green", icon="play")
    ).add_to(m)

    folium.Marker(
        location=[lat2, lon2],
        popup="End Location",
        tooltip="End",
        icon=folium.Icon(color="red", icon="stop")
    ).add_to(m)



for i, row in lat_lon_df.iterrows():
    lat1, lon1, lat2, lon2 = row['lat_start'], row['lon_start'], row['lat_end'], row['lon_end']
    r = get_route_json(lat1, lon1, lat2, lon2)
    if i == 0:
        continue
    if i + 1 == len(lat_lon_df):
        break
    else:
        folium.PolyLine([[r[0][0], r[1][1]], [r[i+1][0], r[i+1][1]]], color='blue', weight=5, opacity=0.7).add_to(m)
    if i + 1 == 12:
        break


m.save('route_map.html')


{'hints': {'visited_nodes.sum': 2, 'visited_nodes.average': 2.0}, 'info': {'copyrights': ['GraphHopper', 'OpenStreetMap contributors'], 'took': 2, 'road_data_timestamp': '2025-02-10T03:00:00Z'}, 'paths': [{'distance': 0.0, 'weight': 0.0, 'time': 0, 'transfers': 0, 'points_encoded': False, 'bbox': [-79.42368, 43.639624, -79.42368, 43.639624], 'points': {'type': 'LineString', 'coordinates': [[-79.42368, 43.639624], [-79.42368, 43.639624]]}, 'instructions': [{'distance': 0.0, 'sign': 4, 'interval': [0, 0], 'text': 'Arrive at destination', 'time': 0, 'street_name': ''}], 'legs': [], 'details': {}, 'ascend': 0.0, 'descend': 0.0, 'snapped_waypoints': {'type': 'LineString', 'coordinates': [[-79.42368, 43.639624], [-79.42368, 43.639624]]}}]}
{'hints': {'visited_nodes.sum': 66, 'visited_nodes.average': 66.0}, 'info': {'copyrights': ['GraphHopper', 'OpenStreetMap contributors'], 'took': 2, 'road_data_timestamp': '2025-02-10T03:00:00Z'}, 'paths': [{'distance': 2638.382, 'weight': 415.211687, 'tim

TypeError: 'NoneType' object is not subscriptable